In [ ]:
import pandas as pd
import numpy as np
import urllib
import urllib.request as request
import matplotlib.pyplot as plt
import seaborn as sns
import json
from bs4 import BeautifulSoup
from github import Github
from github import GithubException, RateLimitExceededException
from config import token as token
import time

In [ ]:
url = 'https://government.github.com/community/'
html_response = request.urlopen(url)
soup = BeautifulSoup(html_response, 'html.parser')

In [ ]:
c1, c2, c3, c4, c5 = [], [], [], [], []
for div in soup.find_all('div', id = lambda x: x and x.startswith('include')):
    subDiv = div.find_all('div', id = lambda x: x and x.startswith('type'))
    for child in subDiv:
        leafDiv = child.find_all('div', class_ = lambda x: x and x.endswith('org'))
        for leaf in leafDiv:
            org = leaf.find_all('a')
            for k in org:
                c1.append(div.get('id').split('-')[1])
                c2.append(child.get('id').split('-')[1])
                c3.append(k.get('title'))
                c4.append(k.get('href'))

In [ ]:
df = pd.DataFrame()
df['Type'] = c1
df['Country/Division'] = c2
df['Name'] = c3
df['Homepage'] = c4

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
l0, l1, l2, l3 = [], [], [], []

In [ ]:
for org in c3:
    try:
        g = Github(token)
        u = g.get_user(org)
        rls = u.get_repos()
        for r in rls:
#             print(org, r, rl.remaining)
            while g.get_rate_limit().core.remaining <= 1:
                print('API access rate limit reached sleeping')
                time.sleep(60)
            l0.append(org)
            l1.append(r.name)
            l2.append(r.language)
            l3.append(r.get_contributors().totalCount)
    except GithubException:
        print('The repo does not have required information')

In [ ]:
df2 = pd.DataFrame()
df2['Country/Division'] = l0
df2['RepoName'] = l1
df2['Language'] = l2
df2['Contributors'] = l3

In [ ]:
df2.shape

In [ ]:
output1 = df[['Country/Division', 'Type']]
output1 = output1.rename(columns={'Country/Division': 'government_name', 'Type': 'repo_count'})

In [ ]:
output1 = output1.groupby('government_name', as_index=False).count()

In [ ]:
output1.head()

In [ ]:
output1.to_csv('gov_repo_count.csv')

In [ ]:
df2.to_csv('gov_repo_spec.csv')